In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(py['main'])
import global_settings,ReadData,ShockFunction,Production,Household,GE,Invest,Trade,Government,diagnostics
from DataBase_wheels import small_updates
from gmspython import gmspython_i
os.chdir(curr)
data_folder = os.getcwd()+'\\Data\\IO'
gams_folder = os.getcwd()+'\\gamsmodels\\GE'

The file_gams_py_gdb0.gdx is still active and was not deleted.
The file_gams_py_gdb1.gdx is still active and was not deleted.
The file_gams_py_gdb3.gdx is still active and was not deleted.


# Set up DCGE model from saved model components

*The current general equilibrium model is a small open economy that features exogenous long run interest-, inflation-, and growth rates. These settings are defined in the global settings:*

In *Example1.ipynb* the different modules of the model was collected and calibrated to a partial equilibrium scenario. Here, we draw on these modules to set up an integrated model. Note that this allows us to flexibly add/remove certain modules, and re-run the model. This can be relevant in terms of investigating policy scenarios, but also for debugging purposes and to help with numerical complications. 

The setup for the model is outlined in *Example1.ipynb*. Here we focus on loading the modules and running the integrated model (in general equilibrium). One difference is the general equilibrium module; the conditions, exogenous/endogenous variables needed here draws on the other modules.

## **1: Load modules**

Load modules:

In [2]:
modules = {'p': Production.pr_dynamic(pickle_path=gams_folder+'\\gmspython_p'),
           'HH': Household.hh_dynamic(pickle_path=gams_folder+'\\gmspython_HH'),
           'inv': Invest.inv_dynamic(pickle_path=gams_folder+'\\gmspython_inv'),
           'itory': Invest.itoryD(pickle_path=gams_folder+'\\gmspython_itory'),
           'trade': Trade.trade_dynamic(pickle_path=gams_folder+'\\gmspython_trade'),
           'G': Government.g_dynamic(pickle_path=gams_folder+'\\gmspython_G')}

Load data:

In [3]:
GE_data = DataBase.GPM_database(pickle_path=gams_folder+'\\GE_data')

## **2: Initialize integrated model**

Initialize *gmspython_i* model and add modules:

In [4]:
gm_i = gmspython_i(work_folder=work_folder,database_kw = {'name': 'db_ex1'},**{'name':'ex1'})
[gm_i.add_module(m) for n,m in modules.items()];

The integrated model *gm_i* adopts the namespaces from other modules, merges the databases into one, and keeps the modules in the *modules* attribute:

In [5]:
gm_i.modules

{'p': <Production.pr_dynamic at 0x22e73954d88>,
 'HH': <Household.hh_dynamic at 0x22e73a56548>,
 'inv': <Invest.inv_dynamic at 0x22e7380d9c8>,
 'itory': <Invest.itoryD at 0x22e739bc188>,
 'trade': <Trade.trade_dynamic at 0x22e73a7c148>,
 'G': <Government.g_dynamic at 0x22e73a78908>}

Finally, we define the equilibrium module from the other modules. The GE version 'v1' is suited for the case with demand/supply being defined over sectors and goods:

In [6]:
gm = GE.GE_v1(work_folder=work_folder,**{'data_folder': gams_folder,'name':'GE_module'})

We initialize the module from the integrated model. The module attemps to figure out which quantities/prices needs to be endogenized in order to make the model square. There are some cases, however, we have to adjust manually. One example is if the supply of a good is exogenously given (as labor is in our simple household model). To adjust for this we add the options that the subset 'qS_endo' (which supply quantities to be endogenized) should not be included. Specifically, we ask not to include the subset 'exo' in the household module:

In [7]:
gm_i.get('exo',module='HH')

MultiIndex([('HH', 'L')],
           names=['s', 'n'])

We initialize the module using these settings, and end with applying the 'write' method (writes the relevant gams code for the module, but does not run anything, as opposed to the write_and_run method) as well as adding this module to the integrated model:

In [8]:
ctree_kwargs = {'qS_endo': {'not': gm_i.g('exo',module='HH')}}
gm.init_from_model_i(gm_i,ctree_kwargs=ctree_kwargs)
gm.write()
gm.setstate('DC') 
gm_i.add_module(gm)

## **3: Run and calibrate**

*Compute the value of the disequilibrium:*

In [9]:
def s(db):
    return db['qS'].rctree_pd({'and': [db['d_qS'],db['n_equi'],db['txE']]}).groupby(['t','n']).sum()
def d(db):
    return db['qD'].rctree_pd({'and': [db['d_qD'],db['n_equi'],db['txE']]}).groupby(['t','n']).sum()
def diseq(db):
    return (s(db)-d(db)).dropna().unstack()

Now that the integrated model *gm_i* has all the relevant modules, we can start by running the baseline model, after merging the 'settings'. Note that we add the option *write=False*, as the integrated model does not need to write any of the gams code: All of it has been written and added via the separate modules:

In [10]:
gm_i.merge_settings()
# gm_i.write_and_run(write=False)

The result is defined in the model instances (w. default name = 'baseline' if nothing else is supplied):

In [11]:
# gm_i.model_instances['baseline'].__dict__

We can now calibrate the model by updating the state of the model to 'DC', reset settings,

In [12]:
gm_i.setstate('DC')

*Time-specific moments:*

In [13]:
GE_t = DataBase.GPM_database()
for var in GE_data.variables_flat:
    GE_t[var] = DataBase_wheels.repeat_variable_windex(GE_data.get(var),gm_i.get('t0'))

*Keep exogenous part:*

In [14]:
GE_t = gm_i.slice_exo(GE_t,copy=False)

TypeError: vS is not aliased

*Calibrate sneakily:*

In [ ]:
gm_i.initialize_variables()
gm_i.setstate('DC')
kwargs_write ={'end': DB2Gams.run_text(g_exo=gm_i.exo_groups.keys(),g_endo=gm_i.endo_groups.keys(),blocks=gm_i.model.settings.get_conf('blocks'),name=gm_i.model.settings.get_conf('name'))}

In [ ]:
gm_i.setstate('B')
gm_i.write_and_run(name='dc',kwargs_write=kwargs_write,write=False,add_checkpoint='dc')
shock_db,kwargs_shock = ShockFunction.sneaky_db(gm_i.model_instances['dc'].out_db,GE_t)
gm_i.model_instances['dc'].solve_sneakily(from_cp=True,cp_init=gm_i.checkpoints['dc'],shock_db=shock_db,kwargs_shock=kwargs_shock,model_name=gm_i.model.settings.conf['DC']['name'])